# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 26 2022 2:17PM,247301,10,MSP214289,"Methapharm, Inc.",Released
1,Sep 26 2022 2:17PM,247300,20,8574235,"Exact-Rx, Inc.",Released
2,Sep 26 2022 2:17PM,247300,20,8574811,"Exact-Rx, Inc.",Released
3,Sep 26 2022 2:17PM,247300,20,8576100,"Exact-Rx, Inc.",Released
4,Sep 26 2022 2:17PM,247300,20,8576101,"Exact-Rx, Inc.",Released
5,Sep 26 2022 2:17PM,247300,20,8576104,"Exact-Rx, Inc.",Released
6,Sep 26 2022 2:17PM,247300,20,8576107,"Exact-Rx, Inc.",Released
7,Sep 26 2022 2:17PM,247300,20,8576109,"Exact-Rx, Inc.",Released
8,Sep 26 2022 2:17PM,247300,20,8576112,"Exact-Rx, Inc.",Released
9,Sep 26 2022 2:17PM,247300,20,8576113,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,247297,Released,24
33,247298,Released,1
34,247299,Released,1
35,247300,Released,10
36,247301,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247297,NaN,NaN,24.0
247298,NaN,NaN,1.0
247299,NaN,NaN,1.0
247300,NaN,NaN,10.0
247301,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247219,0.0,14.0,1.0
247220,0.0,0.0,3.0
247223,0.0,0.0,8.0
247238,0.0,0.0,1.0
247240,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247219,0,14,1
247220,0,0,3
247223,0,0,8
247238,0,0,1
247240,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247219,0,14,1
1,247220,0,0,3
2,247223,0,0,8
3,247238,0,0,1
4,247240,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247219,,14,1
1,247220,,,3
2,247223,,,8
3,247238,,,1
4,247240,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 26 2022 2:17PM,247301,10,"Methapharm, Inc."
1,Sep 26 2022 2:17PM,247300,20,"Exact-Rx, Inc."
11,Sep 26 2022 2:15PM,247299,10,Emerginnova
12,Sep 26 2022 2:13PM,247298,22,"NBTY Global, Inc."
13,Sep 26 2022 2:07PM,247297,19,"Innogenix, LLC"
37,Sep 26 2022 1:57PM,247295,19,"AdvaGen Pharma, Ltd"
61,Sep 26 2022 1:51PM,247294,10,"Citieffe, Inc."
62,Sep 26 2022 1:41PM,247293,20,"Adare Pharmaceuticals, Inc."
63,Sep 26 2022 1:39PM,247292,10,"Nextsource Biotechnology, LLC"
65,Sep 26 2022 1:21PM,247288,16,Sartorius Lab Products and Service


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 26 2022 2:17PM,247301,10,"Methapharm, Inc.",,,1
1,Sep 26 2022 2:17PM,247300,20,"Exact-Rx, Inc.",,,10
2,Sep 26 2022 2:15PM,247299,10,Emerginnova,,,1
3,Sep 26 2022 2:13PM,247298,22,"NBTY Global, Inc.",,,1
4,Sep 26 2022 2:07PM,247297,19,"Innogenix, LLC",,,24
5,Sep 26 2022 1:57PM,247295,19,"AdvaGen Pharma, Ltd",,,24
6,Sep 26 2022 1:51PM,247294,10,"Citieffe, Inc.",,,1
7,Sep 26 2022 1:41PM,247293,20,"Adare Pharmaceuticals, Inc.",,,1
8,Sep 26 2022 1:39PM,247292,10,"Nextsource Biotechnology, LLC",,,2
9,Sep 26 2022 1:21PM,247288,16,Sartorius Lab Products and Service,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 2:17PM,247301,10,"Methapharm, Inc.",1,,
1,Sep 26 2022 2:17PM,247300,20,"Exact-Rx, Inc.",10,,
2,Sep 26 2022 2:15PM,247299,10,Emerginnova,1,,
3,Sep 26 2022 2:13PM,247298,22,"NBTY Global, Inc.",1,,
4,Sep 26 2022 2:07PM,247297,19,"Innogenix, LLC",24,,
5,Sep 26 2022 1:57PM,247295,19,"AdvaGen Pharma, Ltd",24,,
6,Sep 26 2022 1:51PM,247294,10,"Citieffe, Inc.",1,,
7,Sep 26 2022 1:41PM,247293,20,"Adare Pharmaceuticals, Inc.",1,,
8,Sep 26 2022 1:39PM,247292,10,"Nextsource Biotechnology, LLC",2,,
9,Sep 26 2022 1:21PM,247288,16,Sartorius Lab Products and Service,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 2:17PM,247301,10,"Methapharm, Inc.",1,,
1,Sep 26 2022 2:17PM,247300,20,"Exact-Rx, Inc.",10,,
2,Sep 26 2022 2:15PM,247299,10,Emerginnova,1,,
3,Sep 26 2022 2:13PM,247298,22,"NBTY Global, Inc.",1,,
4,Sep 26 2022 2:07PM,247297,19,"Innogenix, LLC",24,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 2:17PM,247301,10,"Methapharm, Inc.",1.0,NaN,NaN
1,Sep 26 2022 2:17PM,247300,20,"Exact-Rx, Inc.",10.0,NaN,NaN
2,Sep 26 2022 2:15PM,247299,10,Emerginnova,1.0,NaN,NaN
3,Sep 26 2022 2:13PM,247298,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Sep 26 2022 2:07PM,247297,19,"Innogenix, LLC",24.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 2:17PM,247301,10,"Methapharm, Inc.",1.0,0.0,0.0
1,Sep 26 2022 2:17PM,247300,20,"Exact-Rx, Inc.",10.0,0.0,0.0
2,Sep 26 2022 2:15PM,247299,10,Emerginnova,1.0,0.0,0.0
3,Sep 26 2022 2:13PM,247298,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Sep 26 2022 2:07PM,247297,19,"Innogenix, LLC",24.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5439910,46.0,15.0,1.0
16,741812,5.0,0.0,0.0
19,989131,50.0,0.0,0.0
20,741855,51.0,11.0,0.0
21,247246,0.0,1.0,0.0
22,247298,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5439910,46.0,15.0,1.0
1,16,741812,5.0,0.0,0.0
2,19,989131,50.0,0.0,0.0
3,20,741855,51.0,11.0,0.0
4,21,247246,0.0,1.0,0.0
5,22,247298,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,46.0,15.0,1.0
1,16,5.0,0.0,0.0
2,19,50.0,0.0,0.0
3,20,51.0,11.0,0.0
4,21,0.0,1.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,46.0
1,16,Released,5.0
2,19,Released,50.0
3,20,Released,51.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21,22
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,15.0,0.0,0.0,11.0,1.0,0.0
Released,46.0,5.0,50.0,51.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,15.0,0.0,0.0,11.0,1.0,0.0
2,Released,46.0,5.0,50.0,51.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,15.0,0.0,0.0,11.0,1.0,0.0
2,Released,46.0,5.0,50.0,51.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()